# Projet de Simulation et Monte Carlo  
  
  Ce projet se base sur l'arcticle de Yuyang Shi et Rob Cornish : On Multilevel Monte Carlo Unbiased Gradient Estimation For Deep Latent Variable Models (2021) http://proceedings.mlr.press/v130/shi21d.html

In [ ]:
import numpy as np
from scipy.stats import multivariate_normal
from scipy.special import logsumexp
import matplotlib.pyplot as plt
import random

## Partie 1 : Génération des données 
Comme dit dans l'article, nous étudions les modèles à variables latentes. pour pouvoir générer les données, nous considérerons une expérience gaussienne linéaire de la même manière que Rainforth et al. (2018) et Tucker et al. (2019)  

   On considère le modèle génératif suivant : $p_{\theta}(x,z)= \mathcal{N}(z|\theta,I)\mathcal{N}(x|z,I)$ où $x,z \in \mathbb{R^{20}}$ tel que $p_{\theta}(x)= \mathcal{N}(x|\theta,2I)$ et $p_{\theta}(z|x)= \mathcal{N}(\frac{\theta+x}{2},\frac{1}{2}I)$. La distribution de l'encodeur est $q_{\phi}(z|x)=\mathcal{N}(z|Ax+b,\frac{2}{3}I)$, où $\phi = (A,b)$. On considère une perturbation aléatoire des paramètres autour de la valeur optimale par une gaussienne centré de variance 0.01.
    

In [ ]:
theta = np.random.normal(0, 1)
theta_vector = np.zeros(20)+theta
r = 0.6
A_true = 1/2*np.eye(20)
b_true = np.zeros(20)+theta/2
X_data = multivariate_normal.rvs(theta_vector, 2*np.eye(20))
print('theta = ' + str(theta))

In [ ]:
def parameter_perturbation(A, b):
    return (A+np.eye(20)*np.random.normal(0, 0.01), b+np.random.normal(0, 0.01))

In [ ]:
def generate_z(K, A, b, x):
    Z = []
    for i in range(2**(K+1)):
        Z.append(multivariate_normal.rvs(np.dot(A,x)+b, 2/3*np.eye(20)))
    Z_O = Z[1::2]
    Z_E = Z[::2]
    return Z,Z_O,Z_E

In [ ]:
def q_phi(z, A, b, K, x):
    q = []
    for i in range(2**(K+1)):
        q.append(multivariate_normal.pdf(z[i], np.dot(A,x)+b, 2/3*np.eye(20)))
    return np.array(q)

In [ ]:
def p_joint(z, K, x, mu):
    p = []
    mu_vector = np.zeros(20)+mu
    for i in range(2**(K+1)):
        p.append(multivariate_normal.pdf(z[i], mu_vector, np.eye(20))*multivariate_normal.pdf(x,z[i],np.eye(20)))
    return np.array(p)

## Partie 2 : Estimation de la vraisemblance  
  
  Dans cette partie, nous allons reproduire les estimateurs IWAE, SS, RR et SUMO cités dans l'article

### 1) Estimateur SS

#### Définition de l'estimateur SS : 

Nous définissions ici l'estimateur SS 

In [ ]:
def SS_estimator(A, b, x, mu, N_sim):
    list_ss = []
    for i in range(N_sim):
        K = np.random.geometric(r)
        Z_data, Z_data_O, Z_data_E = generate_z(K, A, b, x)
        q = q_phi(Z_data, A, b, K, x)
        p = p_joint(Z_data, K, x, mu)
        w = p/q
        log_w = np.log(w)
        log_w_O = log_w[1::2]
        log_w_E = log_w[::2]
        I0 = np.mean(log_w)
        l_O = logsumexp(log_w_O) - np.log(len(log_w_O))
        l_E = logsumexp(log_w_E) - np.log(len(log_w_E))
        l_OUE = logsumexp(log_w) - np.log(len(log_w))
        delta_K = l_OUE - 0.5*(l_O + l_E)
        list_ss.append(I0 + delta_K/(r*(1-r)**(K-1)))
    return np.mean(list_ss)

#### Valeur de l'estimateur SS pour le vrai phi :

In [ ]:
SS_estimator(A_true, b_true, X_data, 1.31,1000)

#### Simulation de plusieurs phi et simulation des estimateurs SS correspondants :

In [ ]:
u = np.linspace(-4,4,100)

A_perturbed, b_perturbed = A_true, b_true
for i in range(10):
    A_perturbed, b_perturbed = parameter_perturbation(A_perturbed, b_perturbed)
    
SS_estim_phisim = []
for i in range(100):
    SS_estim_phisim.append(SS_estimator(A_perturbed, b_perturbed, X_data, u[i],1000))

#### Simulation estimateur Vraissemblance pour les phi simulés :

In [ ]:
def log_likelihood(X_data,mu):
    mu_vector = np.zeros(20)+mu
    return -10*np.log(2*np.pi)-0.5*np.log(4)-0.5*np.dot(np.transpose(X_data-mu_vector),X_data-mu_vector)

In [ ]:
LL_estim_phisim = []
for i in range(100):
    LL_estim_phisim.append(log_likelihood(X_data,u[i]))

#### Visualisation graphique des estimateurs VD et HG :

In [ ]:
plt.plot(u,HG,label='HG')
plt.plot(u,VD,label='VD')
plt.xlabel("phi simulés")
plt.ylabel("estimateurs LL et SS correspondants")
plt.legend(loc='upper right',fontsize='10')

## Estim RR 

In [ ]:
def RR_estimator(A, b, x, mu, N_sim):
    list_rr = []
    for i in range(N_sim):
        Sum=0
        K = np.random.geometric(r)
        for l in range(K+1):
            Z_data, Z_data_O, Z_data_E = generate_z(l, A, b, x)
            q = q_phi(Z_data, A, b, l, x)
            p = p_joint(Z_data, l, x, mu)
            w = p/q
            log_w = np.log(w)
            log_w_O = log_w[1::2]
            log_w_E = log_w[::2]
            I0 = np.mean(log_w)
            l_O = logsumexp(log_w_O) - np.log(len(log_w_O))
            l_E = logsumexp(log_w_E) - np.log(len(log_w_E))
            l_OUE = logsumexp(log_w) - np.log(len(log_w))
            delta_l = l_OUE - 0.5*(l_O + l_E)
            Sum = Sum + delta_l/((1-r)**min(l-1,0))
        list_rr.append(I0 + Sum)
    return np.mean(list_rr)

In [ ]:
RR_estimator(A_true, b_true, X_data, 1.31,1000)

In [ ]:
u = np.linspace(-4,4,100)

A_perturbed, b_perturbed = A_true, b_true
for i in range(10):
    A_perturbed, b_perturbed = parameter_perturbation(A_perturbed, b_perturbed)
    
RR_estim_phisim = []
for i in range(100):
    RR_estim_phisim.append(RR_estimator(A_perturbed, b_perturbed, X_data, u[i],1000))

In [ ]:
def log_likelihood(X_data,mu):
    mu_vector = np.zeros(20)+mu
    return -10*np.log(2*np.pi)-0.5*np.log(4)-0.5*np.dot(np.transpose(X_data-mu_vector),X_data-mu_vector)

LL_estim_phisim = []
for i in range(100):
    LL_estim_phisim.append(log_likelihood(X_data,u[i]))

In [ ]:
plt.plot(u,LL_estim_phisim,label='LL')
plt.plot(u,RR_estim_phisim,label='RR')
plt.xlabel("phi simulés")
plt.ylabel("estimateurs LL et RR correspondants")
plt.legend(loc='upper right',fontsize='10')